In [2]:
import tensorflow as tf
import cv2 as cv
import numpy as np 
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')
! unzip /content/drive/MyDrive/train.csv.zip
! unzip /content/drive/MyDrive/test.csv.zip

Mounted at /content/drive
Archive:  /content/drive/MyDrive/train.csv.zip
  inflating: train.csv               
Archive:  /content/drive/MyDrive/test.csv.zip
  inflating: test.csv                


In [4]:
dataset = pd.read_csv('train.csv')
print(dataset.head())
X_train = dataset.iloc[:,1:].values
y_train= dataset.iloc[:,:1]
dataset1 = pd.read_csv('test.csv')
X_test = dataset1.iloc[:,:].values


   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         

In [5]:
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
# y_train = ohe.fit_transform(y_train)

In [6]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
dummy_y = np_utils.to_categorical(y_train)
print(dummy_y)

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [42]:
x_array = np.asarray(X_train)
y_array = np.asarray(y_train)
x_array = np.asarray(x_array).astype('float32')

In [8]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=10, activation='softmax'))
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, dummy_y, batch_size = 32, epochs = 25)

Epoch 1/25
1313/1313 [==============================] - 3s 2ms/step - loss: 0.3591 - accuracy: 0.8915
Epoch 2/25
1313/1313 [==============================] - 3s 2ms/step - loss: 0.1530 - accuracy: 0.9531
Epoch 3/25
1313/1313 [==============================] - 3s 2ms/step - loss: 0.1177 - accuracy: 0.9632
Epoch 4/25
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0950 - accuracy: 0.9702
Epoch 5/25
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0788 - accuracy: 0.9743
Epoch 6/25
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0652 - accuracy: 0.9793
Epoch 7/25
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0563 - accuracy: 0.9812
Epoch 8/25
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0526 - accuracy: 0.9821
Epoch 9/25
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0456 - accuracy: 0.9844
Epoch 10/25
1313/1313 [==============================] - 3s 2ms/step - loss: 0.039

In [29]:
y_pred = ann.predict(X_test)
y_pred.shape
slno= []
pred= []
for i in range(28000):
  slno.append(i+1)
for i in range(28000):
  pred.append(np.argmax(y_pred[i,:]))
Pred = np.asarray(pred)
Slno = np.asarray(slno)
results = np.concatenate((Slno.reshape(len(Slno),1), Pred.reshape(len(Pred),1)),1)
print(results)

[[    1     2]
 [    2     0]
 [    3     9]
 ...
 [27998     3]
 [27999     9]
 [28000     2]]


In [33]:
np.savetxt("ANN.xlsx", results, delimiter=",")